# SetUP

In [1]:
setwd("/lustre/user/liclab/jialm/Project/SCOPEC/analysis/stripes/res/allhumanBrain_1k_PFC_V1/")

In [2]:
suppressPackageStartupMessages({
library(dplyr)
library(tidyverse)
library(data.table)
library(ggplot2)
library(edgeR)
library(RColorBrewer)
library(GenomicRanges)
library(ComplexHeatmap)
#library(igraph)
})

In [3]:
file.ls <- list.files("./",pattern="sta")

In [5]:
stripes.sta <- lapply(file.ls,read.table)

In [9]:
names(stripes.sta) <- file.ls 

In [17]:
counts <- dplyr::full_join(stripes.sta$`human_1K-PFC_V1_merge.RG.iPC.eN.iN.stripes.PFC.RG.counts.sta`,
                 stripes.sta$`human_1K-PFC_V1_merge.RG.iPC.eN.iN.stripes.V1.RG.counts.sta`,
                 by=paste0("V",1:6)) %>%
dplyr::full_join(.,stripes.sta$`human_1K-PFC_V1_merge.RG.iPC.eN.iN.stripes.PFC.iPC.counts.sta`,by=paste0("V",1:6))%>%
dplyr::full_join(.,stripes.sta$`human_1K-PFC_V1_merge.RG.iPC.eN.iN.stripes.V1.iPC.counts.sta`,by=paste0("V",1:6))%>%
dplyr::full_join(.,stripes.sta$`human_1K-PFC_V1_merge.RG.iPC.eN.iN.stripes.PFC.eN.counts.sta`,by=paste0("V",1:6))%>%
dplyr::full_join(.,stripes.sta$`human_1K-PFC_V1_merge.RG.iPC.eN.iN.stripes.V1.eN.counts.sta`,by=paste0("V",1:6))%>%
dplyr::full_join(.,stripes.sta$`human_1K-PFC_V1_merge.RG.iPC.eN.iN.stripes.PFC.iN.counts.sta`,by=paste0("V",1:6))%>%
dplyr::full_join(.,stripes.sta$`human_1K-PFC_V1_merge.RG.iPC.eN.iN.stripes.V1.iN.counts.sta`,by=paste0("V",1:6)) %>%
replace(is.na(.),0) %>%
set_names(c(paste0("V",1:6),"RG.1","RG.2","iPC.1","iPC.2","eN.1","eN.2","iN.1","iN.2"))

In [101]:
##RGToeN

data <- Inpt[,c(1,2,5,6)] %>% set_names("RG.1","RG.2","eN.1","eN.2")
group <- c(rep("RG",2),rep("eN",2))
genes <- counts[,1:6]
dge <- DGEList(counts = data, genes = genes, group = group)
#keep <- filterByExpr(dge, group = group)
dge <- calcNormFactors(dge)
#dge <- dge[keep,]
dge <- estimateDisp(dge)
design <- model.matrix(~ 0 + group, data = dge$samples)
fit <- glmQLFit(dge, design = design)
AvsB <- makeContrasts(groupeN-groupRG, levels=design)
qlf <- glmQLFTest(fit, contrast= AvsB)

RGtoeN <- topTags(qlf, n = Inf)$table %>% mutate(Group=case_when(logFC >=0.5 & FDR<=0.05~"Up",
                                                       logFC <= -0.5 & FDR<=0.05~"Down",
                                                        abs(logFC) <0.5 | FDR>0.05~"Not"))
RGtoeN %>% group_by(Group) %>% summarise(n=n())

Using classic mode.



Group,n
<chr>,<int>
Down,2260
Not,23345
Up,2629


In [124]:
##eNToOthers

data <- Inpt[,c(5,6,1:4,7,8)] %>% set_names("eN.1","eN.2","other.1","other.2","other.3","other.4","other.5","other.6")
group <- c(rep("eN",2),rep("other",6))
genes <- counts[,1:6]
dge <- DGEList(counts = data, genes = genes, group = group)
#keep <- filterByExpr(dge, group = group)
dge <- calcNormFactors(dge)
#dge <- dge[keep,]
dge <- estimateDisp(dge)
design <- model.matrix(~ 0 + group, data = dge$samples)
fit <- glmQLFit(dge, design = design)
AvsB <- makeContrasts(groupeN-groupother, levels=design)
qlf <- glmQLFTest(fit, contrast= AvsB)

eNtoOther <- topTags(qlf, n = Inf)$table %>% mutate(Group=case_when(logFC >=0.5 & FDR<=0.05~"Up",
                                                       logFC <= -0.5 & FDR<=0.05~"Down",
                                                        abs(logFC) <0.5 | FDR>0.05~"Not"))
eNtoOther %>% group_by(Group) %>% summarise(n=n())

Using classic mode.



Group,n
<chr>,<int>
Down,517
Not,26963
Up,754


In [128]:
##RGToOthers

data <- Inpt %>% set_names("RG.1","RG.2","other.1","other.2","other.3","other.4","other.5","other.6")
group <- c(rep("RG",2),rep("other",6))
genes <- counts[,1:6]
dge <- DGEList(counts = data, genes = genes, group = group)
#keep <- filterByExpr(dge, group = group)
dge <- calcNormFactors(dge)
#dge <- dge[keep,]
dge <- estimateDisp(dge)
design <- model.matrix(~ 0 + group, data = dge$samples)
fit <- glmQLFit(dge, design = design)
AvsB <- makeContrasts(groupRG-groupother, levels=design)
qlf <- glmQLFTest(fit, contrast= AvsB)

RGtoOther <- topTags(qlf, n = Inf)$table %>% mutate(Group=case_when(logFC >=0.5 & FDR<=0.1~"Up",
                                                       logFC <= -0.5 & FDR<=0.1~"Down",
                                                        abs(logFC) <0.5 | FDR>0.1~"Not"))
RGtoOther %>% group_by(Group) %>% summarise(n=n())

Using classic mode.



Group,n
<chr>,<int>
Down,385
Not,27622
Up,227


In [121]:
write.table(RGtoeN,"./all.humanBrain.eNtoRG.PFCV1.1k.stripes_20_50_S+L.edgeR.FDR0.05_log2FC0.5.res",
            col.names = F,row.names = F,sep="\t",quote = F)

In [129]:
write.table(eNtoOther,"./all.humanBrain.eNtoOthers.PFCV1.1k.stripes_20_50_S+L.edgeR.FDR0.05_log2FC0.5.res",
            col.names = F,row.names = F,sep="\t",quote = F)

In [130]:
write.table(RGtoOther,"./all.humanBrain.RGtoOthers.PFCV1.1k.stripes_20_50_S+L.edgeR.FDR0.1_log2FC0.5.res",
            col.names = F,row.names = F,sep="\t",quote = F)